In [24]:
import tensorflow as tf
import pickle
import keras
from keras.models import Model, load_model, clone_model
from keras.utils import multi_gpu_model
from keras.layers import *
from keras.optimizers import Adam, SGD
from keras.regularizers import l2
from keras.losses import binary_crossentropy
import keras.backend as K
import numpy as np
import matplotlib.pyplot as plt
import sys
import random
sys.path.append('src')  # Fix for jupyter
import src.emulator as emulator
import src.emulator_utils as emulator_utils
import src.emulator_vis as emulator_vis
%matplotlib inline
from IPython.display import clear_output
import time
import multiprocessing
from multiprocessing import Event, Queue, Pipe
from multiprocessing import Process as Thread
import os
import logging
from mcts import *
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Logging

In [2]:
logging.basicConfig(filename='logging.log', level=logging.DEBUG, format='%(asctime)s %(message)s', filemode='w')

# Model Params

In [3]:
INPUT_SIZE = (16, 16, 5) # Map size fixed to 16x16 (2 to 3 players)
N_ACTIONS = 4
gpus = 1

# Define the Layers Blocks

In [4]:
filters = 48

# Convolutional Block
def conv_block(in_layer, name, filters=filters, kernel_size=(3,3), bn=True, relu=True):
    l = Conv2D(filters, kernel_size, padding='same', name = name, kernel_regularizer=l2(1e-4),
              kernel_initializer='truncated_normal')(in_layer)
    if bn:
        l = BatchNormalization(axis=3, name = name + '_bn')(l)
    if relu:
        l = Activation('relu', name = name + '_relu')(l)
    
    l = Dropout(0.3)(l)
    return l

# Residual Block
def residual_conv(in_layer, idx, filters=filters, kernel_size=(3,3), bn=True, relu=True):
    name = 'res_' + str(idx)
    # Full conv block of pre-defined shape
    l = conv_block(in_layer, name + '_conv1', filters, kernel_size=(3,3), bn=True, relu=True)
    # Second block with skip connection
    l = Conv2D(filters, kernel_size, padding='same', name = name + '_conv2', kernel_regularizer=l2(1e-4),
              kernel_initializer='truncated_normal')(l)
    if bn:
        l = BatchNormalization(axis=3, name = name + '_conv2_bn')(l)
    l = Concatenate()([in_layer, l]) # Skip conn.
    if relu:
        l = Activation('relu', name = name + '_relu')(l)
        
    l = Dropout(0.3)(l)
    return l

def value_head(in_layer):
    l = conv_block(in_layer, 'value_head', filters=1, kernel_size=(1,1))
    l = Flatten(name = 'value_flatten')(l)
    l = Dense(64, name = 'value_dense',
             kernel_initializer='truncated_normal')(l)
    l = Activation('relu', name = 'value_relu')(l)
    l = Dropout(0.3)(l)
    l = BatchNormalization(axis=1, name = 'value_bn')(l)

    l = Dense(1, name = 'value', activation='tanh')(l) # Value output
    return l

def policy_head(in_layer):
    l = conv_block(in_layer, 'policy_head', filters=2, kernel_size=(1,1))
    l = Flatten(name = 'policy_flatten')(l)
    l = Dense(64, name='policy_dense', kernel_initializer='truncated_normal')(l)
    l = Dropout(0.3)(l)
    l = BatchNormalization(axis=1, name = 'policy_bn')(l)
    
    l = Dense(N_ACTIONS, name = 'policy', activation=None,
              kernel_initializer='truncated_normal')(l) # Policy output
    return l

# Model

In [5]:
def create_model():
    def declare_model():
        n_residual = 1#5

        input_layer = Input(INPUT_SIZE)
        l = conv_block(input_layer, 'conv')
        for i in range(n_residual):
            l = residual_conv(l, idx=i + 1)

        policy = policy_head(l)
        
        value = value_head(l)

        alphabot = Model(input_layer, [policy, value])
        return alphabot
    
    if gpus > 1:
        with tf.device('/cpu:0'):
            alphabot = declare_model()
        alphabot_multi = multi_gpu_model(alphabot, gpus=gpus)
        return alphabot_multi, alphabot
    
    alphabot = declare_model()
    return alphabot, alphabot

In [6]:
def softmax(z):
    assert len(z.shape) == 2
    s = np.max(z, axis=1)
    s = s[:, np.newaxis] # necessary step to do broadcasting
    e_x = np.exp(z - s)
    div = np.sum(e_x, axis=1)
    div = div[:, np.newaxis]
    return e_x / div

In [8]:
def simulate_games():
    logging.debug('Starting Threads for parallel Games')
    
    parallel_sim() # Parallel Games
    while not history_buffer.full():
        indices, states = [], []
        if processable_buffer.qsize() < num_threads * 2: # Wait until a bunch of requests are queued
            continue

        for i in range(processable_buffer.qsize()):
            index, state = processable_buffer.get()
            indices.append(index)
            states.append(state)
            
        states = np.array(states, dtype=np.float64)
        #print(states[:, :, :, 0])
        predictions = alphabot.predict(states)
        #print(predictions)
        for i, pred in enumerate(tuple(zip(predictions[0], predictions[1]))):
            pipes[indices[i]].send(dict(zip(alphabot.output_names, pred)))
        # We have to predict until buffer is full
    logging.info('Finished Simulating %s games', n_games)

In [9]:
def play_eval(log_game=False):
    game = emulator.Game(n_players) # TODO: Wrap the following lines in a function
    gmap = game.map # Access map manually on first step

    gmap_old = None # First frame has no older map
    p_alive = game.players_alive # Players alive
    n_alive = game.count_alive()
    
    #maps = [] # Initialise buffer for log
    #maps.append(copy.copy(gmap))
    
    while True:
        assert n_alive == 2, 'Multi player eval is not implemented yet'
        state = map_to_state(gmap, gmap_old, p_alive) # State for each player alive
        
        # The predictions from the candidate and the best bot
        p0 = alphabot.predict(state[0][np.newaxis])
        p1 = alphabot_best.predict(state[1][np.newaxis])
        
        # Split in value and policy
        candidate_policy = p0[0]
        candidate_value = p0[1]
        best_policy = p1[0]
        best_value = p1[1]
        
        logging.debug('Candidate Policy: %s Candidate Value: %s', candidate_policy, candidate_value)
        logging.debug('Best Bot Policy: %s Best Bot Value: %s', best_policy, best_value)
        
        policy = [candidate_policy[0], best_policy[0]]
        policy = softmax(np.array(policy)) # We softmax the policy logits
        chosen_action = np.argmax(policy, axis=-1)
        
        gmap_old = copy.copy(gmap)
        gmap, p_alive, n_alive, reward, game_end = game.step(chosen_action)
        #maps.append(copy.copy(gmap))
        
        if game_end:
            if sum(reward == -1) == 2:
                return play_eval()
            winner = np.where(np.array(p_alive) == 1)[0][0]
            if log_game:
                return maps

            return winner

In [10]:
def train_model():
        # Get a BATCH_SIZE of games
        picked_data = random.sample(complete_history, k=min(BATCH_SIZE, len(complete_history)))
        # Get a State from each game selected
        x = np.empty((len(picked_data), 16, 16, 4), dtype=np.float64)
        actions_taken = []
        rewards = []
        for j, game in enumerate(picked_data):
            index = np.random.randint(0, len(game.actions_taken)) # Get game length and generate index
            action = game.actions_taken[index]
            state = game.states[index]
            #state, action = simmetries(state, action) # Apply random simmetry
            x[j] = np.array(state, dtype=np.float64)
            actions_taken.append(action)    
            rewards.append(np.array(game.rewards[-1], dtype=np.float64))
        
        #rewards = np.array(rewards)
        actions_taken = np.array(actions_taken)
        y = alphabot.predict(x)
        #y = [np.zeros((x.shape[0], 4)), np.zeros((x.shape[0], 1))]
        logging.debug('The predict is %s', y)
        
        for idx, action in enumerate(actions_taken):
            if rewards[idx] == -1: # Loss
                #y[0][idx, :] = 1 # Every other actions is good
                y[0][idx, action] = 0 # Selected is bad
            else: # Win
                y[0][idx, action] = 1 # Every action is bad except this one
            y[1][idx, 0] = rewards[idx] # Policy is easier to manage
        logging.debug('The label is %s', y)
        losses = alphabot.train_on_batch(x, y)
        return losses

In [11]:
def training_cycle():
    global alphabot
    global alphabot_best
    global total_improv
    
    # Simulate n_games (exception made by first interaction)
    logging.info('Starting Training Cycle')
    while len(complete_history) < k * n_games:
        simulate_games()
        # history_buffer contains the games, we store them inside complete history    
        for g in range(history_buffer.qsize()):
            complete_history.append(history_buffer.get())
        stop_simulation() # We can now stop the simulation (will free the memory)
    logging.debug('Complete history should be full, it contains %s elements', len(complete_history))
    # Now we are ready for the training process
    logging.info('Starting Model Training')
    losses = [None, None, None] # For debug purpose
    sum_loss = 0
    cc = 1
    for i in range(t_steps + 1):
        if i % 100 == 0:
            logging.info('Training Interaction: %s losses: %s', i, 
                         round(sum_loss / cc, 2)) # Works?

        losses = train_model()
        sum_loss += losses[0]
        logging.debug('Losses: %s', losses)
        
        improved = False
        evalued_step = False
        cc += 1
        if i % eval_steps == 0 and i > 0:
            evalued_step = True
            cc = 1 # Reset loss counter
            sum_loss = 0
            wins = {'candidate' : 0, 'best' : 0}
            n_c = {0 : 'candidate', 1 : 'best'}
            
            logging.info('Starting self-play evaluation')    
            for j in range(eval_games):
                # 0 is Candidate, 1 is the (soon to be old) best
                wins[n_c[play_eval()]] += 1 # add a win to the winner
                if j % 100 == 0:
                    logging.info('Win state Candidate: %s Best: %s', wins['candidate'], wins['best'])
            win_ratio = wins['candidate'] / eval_games
            if win_ratio > win_percent:
                logging.info('Great! Our candidate won %s percent of games', round(win_ratio * 100, 2))
                total_improv += 1
                logging.info('Our bot got better %s times', total_improv)
                improved = True
                with open(r"alphabot_best.pickle", "wb") as output_file:
                    pickle.dump(alphabot, output_file)
                with open(r"alphabot_best.pickle", "rb") as input_file:
                    alphabot_best = pickle.load(input_file)
            else:
                logging.info('Damn! Our candidate only won %s percent of games', round(win_ratio * 100, 2))            
        if not improved and evalued_step:
            logging.info('Not improved, cloning to best')
            with open(r"alphabot_best.pickle", "rb") as input_file:
                alphabot = pickle.load(input_file)
            
    del complete_history[:n_games] # Delete n oldest games from history

In [12]:
def load_best(best_model):
    global alphabot
    global alphabot_best
    alphabot_best = load_model(best_model)
    alphabot.set_weights(alphabot_best.get_weights())

In [13]:
def train(cycles, best_model = None):
    global alphabot_best
    global alphabot
    
    if best_model != None:
        with open(r"alphabot_best.pickle", "rb") as input_file:
            alphabot_best = pickle.load(input_file)
        with open(r"alphabot_best.pickle", "rb") as input_file:
            alphabot = pickle.load(input_file)
    else:
        with open(r"alphabot_best.pickle", "rb") as input_file:
            alphabot_best = pickle.load(input_file)

    pre_train()
    with open(r"alphabot_best.pickle", "wb") as output_file:
        pickle.dump(alphabot, output_file)
    
    with open(r"alphabot_best.pickle", "rb") as input_file:
        alphabot_best = pickle.load(input_file)
    
    complete_history = []
    for i in range(cycles):
        training_cycle()

In [14]:
class GameRecorder():
    def __init__(self):
        self.states = []
        self.rewards = []
        self.actions_taken = []
        
    def store(self, state, reward, action_taken):
        self.states.append(state)
        self.rewards.append(reward)
        self.actions_taken.append(action_taken)

In [31]:
def ask_predict(idi, x):
    # Adds to queue id and data from process
    processable_buffer.put((idi, x))

def sim(process_id, pipe):
    np.random.seed()
    random.seed()
    
    while True:
        train_steps = simulate_game(MCTS_steps, MCTS_alpha, alphabot, pipe, ask_predict, process_id)    
        
        try:
            for step in train_steps:
                history_buffer.put_nowait(step)
        except:
            break
                    
def stop_simulation():
    global workers
    global history_buffer
    global processable_buffer
    global pipes
    global child_pipes
    
    if 'workers' in globals() and len(workers) != 0:
        for worker in workers:
            worker.terminate()
    workers = []
    
    for pipe in pipes:
        pipe.close()

    for pipe in child_pipes:
        pipe.close()
    
    #for _ in range(history_buffer.qsize()):
    #    try:
    #        history_buffer.get_nowait()
    #    except:
    #        break
            
    #for _ in range(processable_buffer.qsize()):
    #    try:
    #        processable_buffer.get_nowait()
    #    except:
    #        break
        
    history_buffer.close()
    processable_buffer.close()
    
    # Then we empty the queues
    del history_buffer
    del processable_buffer
    del pipes
    del child_pipes

def parallel_sim():
    global workers
    global history_buffer
    global processable_buffer
    global pipes
    global child_pipes
    
    if 'workers' in globals() and len(workers) != 0:
        stop_simulation()
    
    history_buffer = Queue(n_games) # This numbers can be tweaked
    processable_buffer = Queue(num_threads)
    pipes = []
    child_pipes = []
    
    workers = []
    for i in range(num_threads):
        parent_pipe, child_pipe = Pipe() # Pipe to communicate with childs
        pipes.append(parent_pipe)
        child_pipes.append(child_pipe)
        
        worker = Thread(target=sim, args=[i, child_pipe])
        worker.daemon = False
        worker.start()
        workers.append(worker)

In [12]:
def simmetries(state, action):
    # There are these simmetries:
    # +90;+180:-90;-180 degrees rotations
    # Flips
    
    def rotate_state(state, rot):
        N = INPUT_SIZE[0] - 1
        if rot == 0: # Rotation of 0 is simple
            return state
        
        new_state = np.empty(INPUT_SIZE, dtype=np.int)
        it = np.nditer(state, flags=['multi_index'])
        
        if rot == 3: # Rot of 90
            while not it.finished:
                x, y, c = it.multi_index
                new_state[x, y, c] = state[y, N - x, c]
                it.iternext()
                
        elif rot == 2: # Rot of 180
            while not it.finished:
                x, y, c = it.multi_index
                new_state[x, y, c] = state[N - x, N - y, c]
                it.iternext()
        elif rot == 1: # Rot of 270
            while not it.finished:
                x, y, c = it.multi_index
                new_state[x, y, c] = state[N - y, x, c]
                it.iternext()
        return new_state
    
    # First we apply a random rotation simmetry
    simmetry = random.sample([0, 1, 2, 3], 1)[0]
    #print(simmetry * 90)
    #print(action, simmetry)
    action = (action + simmetry) % N_ACTIONS
    state = rotate_state(state, simmetry)
    return state, action

In [13]:
alphabot, _ = create_model()
alphabot.compile(optimizer=Adam(1e-4), 
                          loss={'value' : 'mse', 'policy' : 'categorical_crossentropy'},
                          loss_weights={'value' : 0.1, 'policy' : 2.0})
alphabot.summary(line_length=112)

with open(r"alphabot_best.pickle", "wb") as output_file:
    pickle.dump(alphabot, output_file)
    
with open(r"alphabot_best.pickle", "rb") as input_file:
    alphabot_best = pickle.load(input_file)

________________________________________________________________________________________________________________
Layer (type)                        Output Shape             Param #       Connected to                         
input_1 (InputLayer)                (None, 16, 16, 5)        0                                                  
________________________________________________________________________________________________________________
conv (Conv2D)                       (None, 16, 16, 48)       2208          input_1[0][0]                        
________________________________________________________________________________________________________________
conv_bn (BatchNormalization)        (None, 16, 16, 48)       192           conv[0][0]                           
________________________________________________________________________________________________________________
conv_relu (Activation)              (None, 16, 16, 48)       0             conv_bn[0][0]        

In [21]:
# History of games for training
complete_history = []

# Game Params
n_players = 2
n_games = 15_000 # Simulate N games before each training
k = 6 # Games to be stored n_games * K

# Simulation Params
num_threads = 6

MCTS_steps = 20
MCTS_alpha = 0.8

# Training Params
t_steps = 2000 # Steps of training
eval_steps = 1000 # How many steps before evaluation
eval_games = 500 # How many games to play to evaluate how's best model
win_percent = 0.55 # Ratio of game won to become best model
BATCH_SIZE = 384
total_improv = 0
pretrain_steps = 0 #750
pretrain_games = 0 #30_000

In [19]:
complete_history = []
cycles = 1000
assert False
train(cycles)

AssertionError: 

In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
